In [1]:
from gp.fitgp import GPFitter
from gp.deepgp import DeepGPModel
import pynvml
import torch
from torch.utils.data import DataLoader, TensorDataset
import time
import numpy as np
import matplotlib.pyplot as plt
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo, nvmlShutdown

# Initialize NVML
nvmlInit()
handle = nvmlDeviceGetHandleByIndex(0)  # Assuming GPU 0
def gpu_memory_usage():
    """
    Assumes only one GPU is available.
    Uses the NVIDIA Management Library to return GPU memory usage information.
    :return: Returns the total amount of VRAM available on the GPU and the VRAM usage.
    """
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)  # Assuming you have only one GPU
    mem_info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    total_memory = mem_info.total  # in bytes
    used_memory = mem_info.used  # in bytes
    pynvml.nvmlShutdown()
    return total_memory / (1024 ** 3), used_memory / (1024 ** 3)  # Convert bytes to gigabytes

# Define a function to create and train DeepGPModel on GPU
def benchmark_deepgp_gpu(dataset_sizes, inducing_points_list, dim=4):
    time_results = np.zeros((len(dataset_sizes), len(inducing_points_list)))
    memory_results = np.zeros((len(dataset_sizes), len(inducing_points_list)))

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for i, dataset_size in enumerate(dataset_sizes):
        train_x = torch.randn(dataset_size, dim, device=device)
        train_y = torch.randn(dataset_size, 1, device=device)

        for j, num_inducing_points in enumerate(inducing_points_list):
            model = DeepGPModel(
                train_x_shape=train_x.shape,
                hidden_layers_config=[{'output_dims': None, 'mean_type': 'constant'}],  # Example config
                num_inducing_points=num_inducing_points
            ).to(device)

            train_dataset = TensorDataset(train_x, train_y)
            train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

            # Profile memory usage during training
            torch.cuda.reset_peak_memory_stats()
            torch.cuda.synchronize()

            start_time = time.time()
            GPFitter()(model,
                       train_x,
                       train_y,
                       "deep_gp",
                        batch_size=128,
                       num_epochs=1)
            for epoch in range(10):
                for x_batch, y_batch in train_loader:
                    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
                    loss = torch.nn.functional.mse_loss(model(x_batch), y_batch)
                    loss.backward()
                    # Update parameters

            torch.cuda.synchronize()
            end_time = time.time()
            execution_time = end_time - start_time

            memory_usage = torch.cuda.max_memory_allocated() / 1024 ** 2  # Peak memory usage in MiB

            # Record results
            time_results[i, j] = execution_time
            memory_results[i, j] = memory_usage

    return time_results, memory_results

# Example usage
dataset_sizes = [100, 500, 1000]  # Example dataset sizes
inducing_points_list = [64, 128, 256]  # Example inducing points
time_results, memory_results = benchmark_deepgp_gpu(dataset_sizes, inducing_points_list)

# Shutdown NVML
nvmlShutdown()

# Plotting results (time)
plt.figure(figsize=(10, 6))
for j, num_inducing_points in enumerate(inducing_points_list):
    plt.plot(dataset_sizes, time_results[:, j], marker='o', label=f'Inducing Points: {num_inducing_points}')
plt.xlabel('Dataset Size')
plt.ylabel('Execution Time (seconds)')
plt.title('Execution Time vs. Dataset Size for Different Inducing Points')
plt.legend()
plt.grid(True)
plt.show()

# Plotting results (memory)
plt.figure(figsize=(10, 6))
for j, num_inducing_points in enumerate(inducing_points_list):
    plt.plot(dataset_sizes, memory_results[:, j], marker='o', label=f'Inducing Points: {num_inducing_points}')
plt.xlabel('Dataset Size')
plt.ylabel('Memory Usage (MiB)')
plt.title('Peak Memory Usage vs. Dataset Size for Different Inducing Points')
plt.legend()
plt.grid(True)
plt.show()

AttributeError: 'MultivariateNormal' object has no attribute 'size'